# サンプル
まずはライブラリから必要なものをインポートします

In [2]:
import datetime

from scraping.netkeiba import RaceidScraper, OddsScraper, DatabaseScraper

# BeautifulSoupを利用したクラスの利用方法
このライブラリには以下の2種類のクラスが存在します。
1. requests + BeautifulSoup4を利用した、静的サイトのスクレイピングを行うクラス
1. seleniumを利用した、動的サイトのスクレイピングを行うクラス

netkeibaのスクレイピングにおいては、`RaceidScraper`クラスおよび`DatabaseScraper`クラスが静的サイト用、`OddsScraper`クラスが動的サイト用です。

## 静的サイトのスクレイピング
静的サイトの場合には、クラスの初期化にあたっては何もする必要はありません。

In [3]:
raceid_scraper = RaceidScraper()
race_id_list = raceid_scraper.get_raceID_list_from_date(datetime.date(2021, 12, 11))
race_id_list[:3]

['202107060312', '202109060312', '202107060308']

ただし、DatabaseScraperのように、レースIDを用いてアクセスするURLを変える場合には、まず初めてに`get_soup`メソッドで情報を取得する必要があります。

In [4]:
database_scraper = DatabaseScraper()
database_scraper.get_soup(race_id_list[0])

In [5]:
df = database_scraper.get_main_df()
df.head(3)

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師
0,1,6,10,カーテンコール,牝4,55,川須栄彦,1:11.3,,4.1,1,502(+8),[西]高野友和
1,2,2,3,ミエノワールド,牡4,57,ルメール,1:11.8,3,4.2,2,504(-4),[西]西園正都
2,3,3,5,ビートマジック,セ5,57,藤岡康太,1:12.2,2,47.6,11,478(0),[西]小林真也


In [6]:
df['horse_id'] = database_scraper.get_horse_id_list()
df['jockey_id'] = database_scraper.get_jockey_id_list()
df.head(3)

,着順,枠番,馬番,馬名,性齢,斤量,騎手,タイム,着差,単勝,人気,馬体重,調教師,horse_id,jockey_id
0,1,6,10,カーテンコール,牝4,55,川須栄彦,1:11.3,,4.1,1,502(+8),[西]高野友和,2017102371,2017102371
1,2,2,3,ミエノワールド,牡4,57,ルメール,1:11.8,3,4.2,2,504(-4),[西]西園正都,2017105306,2017105306
2,3,3,5,ビートマジック,セ5,57,藤岡康太,1:12.2,2,47.6,11,478(0),[西]小林真也,2016105963,2016105963


In [7]:
database_scraper.get_pay_df().head(3)

,0,1,2,3
0,単勝,10,410,1
1,複勝,10br3br5,180br190br910,1br3br11
2,枠連,2 - 6,730,3


In [8]:
database_scraper.get_race_info()

{'レース名': '豊川特別 ',
 '周回方向': '不明',
 'コースタイプ': 'ダート',
 '馬場状態': '良',
 'コース長': '1200',
 '天候': '曇',
 '日時': '2021年12月11日',
 '発走': '16:10',
 '開催回': '6回',
 '開催日': '3日目',
 '開催会場': '中京',
 'レース条件_1': '3歳以上2勝クラス',
 'レース条件_2': '(混)(定量)'}

## 動的サイトのスクレイピング
動的サイトの場合にはchrome driverが必要なので、driverの場所を引数から渡す必要があります。

また、静的サイトと同様に、レースIDを用いてアクセスするURLを変える場合には、`visit_page`メソッドを事前に呼び出す必要があります。

In [9]:
odds_scraper = OddsScraper(executable_path='../chromedriver_win32/chromedriver.exe')
odds_scraper.visit_page(race_id_list[0])

In [10]:
tansho_df = odds_scraper.get_tansho_odds_table()
tansho_df.head(3)

,First,Odds
0,1,106.1
1,2,10.7
2,3,4.2
